# Représentations de l'épidémie de COVID-19 en France après été 2020

Pour contribuer ou lire le code de cette petite application : https://foss.heptapod.net/graphics/plot-covid19-fr

## Les données utilisées

Santé publique France fournit des données publiques sur la situation épidémique en France.

-  Données Système d’Informations de DEPistage (SI-DEP) https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/

   Résultats des tests PCR, national et par départements, avec décomposition en classes d'âges.

- Données hospitalières https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/


In [ ]:
import numpy as np
import ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
%matplotlib widget

In [ ]:
from plot_covid19_fr.departements import DEPARTMENTS

In [ ]:
from plot_covid19_fr.faster_dyn_nb import PlotFasterDyn
from plot_covid19_fr.incidence_versus_tests_nb import StatePlotIncidenceVersusTests
from plot_covid19_fr.summary_1loc_nb import Summary1loc

## France : taux de positivité, incidence et situation à l'hôpital (moyenne sur 7 jours)

On trace d'abord le taux de positivité sur 7 jours (figure de gauche), c'est-à-dire le pourcentage de tests positifs sur 7 jours par rapport au nombre total de tests sur ces mêmes 7 jours.
Un des facteurs les plus importants pour prédir la gravité du COVID-19 est l'âge. En dessous de 40 ans, même s'il y a quelques cas graves ou invalidant sur le long terme, la proportion de décés est très faible. Cette proportion augmente ensuite jusqu'à atteindre des valeurs importantes. On trace donc le taux de positivité pour différentes classes d'âges.
Pour augmenter la lisibilité des graphiques, on fait ici le choix de rassembler les classes d'âges 70-79, 80-89 et >=90 en une classe d'âges >=70. Si l'épidémie de COVID-19 se transmet jusqu'à cette classe d'âge, on sait qu'il y a une forte probabilité d'avoir beaucoup de monde aux urgences et un nombre important de décés.

On voit que l'épidémie est largement porté par les "jeunes adultes" avec les courbes verte (20-29 ans), orange (10-19 ans) et rouge (30-39 ans) au dessus de la moyenne. Le taux de positivité des autres classes d'âges adultes augmente exponentiellement avec des périodes de doublement d'environ 15 jours début septembre.

Sur la figure centrale en haut, les mêmes données sont représentées en terme de taux d'incidence, qui est le nombre de tests positifs sur 7 jours pour 100000 habitants. Cette normalisation du nombre de cas positifs fait que l'incidence dépend directement du nombre de tests. Par exemple, si on ne teste pas, le taux d'incidence est égal à 0 même avec une très forte épidémie. On peut donc interpréter cette quantité comme une borne inférieure du nombre de cas pour 100000 habitants (le taux d'incidence "réel"). Par exemple, un taux d'incidence de 100 signifie que au moins 1 personnes sur 1000 est porteuse du virus.
Ainsi, si un taux d'incidence de 100 est grand en terme de dynamique de l'épidémie (la probabilité d'être contaminée doit être reliée au taux d'incidence réel), cette valeur est faible en terme d'immunité de groupe. En effet, il faut en gros 500 semaines à un taux d'incidence réel de 100 pour que 50% de la population ait été touchée.

In [ ]:
summary = Summary1loc()

In [ ]:
summary.widget_yscale

In [ ]:
summary.plot()

Sur la figure centrale en bas, le nombre de tests par jour est représenté. On voit que le nombre de tests a tendance à baisser sur les derniers jours. C'est en partie dû au retard à la remontée des résultats : dans les départements les plus atteints par l'épidémie, les résultats arrivent avec des délais supérieurs à 48h.
Ainsi, il faut considérer les derniers points (aux alentours et à droite des lignes pointillées verticales) avec beaucoup de précaution.

Même en tenant compte de ces retards et donc du fait que ces derniers points sont plus bas qu'ils ne seront après consolidations des résultats, il semble que l'augmentation du taux d'incidence ralentisse.

Les différentes couleurs dans les figures centrales correspondent aux classes d'âges. On voit à partir du 7 septembre une augmentation significative du nombre de tests des 0-9 ans (dans certains départements, voir plus bas, on voit clairement cette augmentation chez les 10-19 ans et les 30-39 ans). Cela pourrait correspondre à des tests d'enfants avec des symptomes évocateurs du COVID-19 (et peut être de leurs parents pour les 30-39 ans). Ces tests peuvent avoir tendance à faire baisser les taux de positivité des classes d'âges concernées et du taux de positivité moyen.

## Carte du taux d'incidence (métropole)

On peut cliquer sur un département pour avoir plus d'informations.

In [ ]:
import ipyleaflet as ipyl
import ipywidgets as ipyw

In [ ]:
from plot_covid19_fr.cartes import data_geojson_dep, get_incidence, get_positivity, color_dep, cmap

In [ ]:
m = ipyl.Map(center=(47.5, 2), zoom=6,
             layout=ipyw.Layout(width='1200px', height='670px'))

geo_json = ipyl.GeoJSON(
    data=data_geojson_dep,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=color_dep
)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))
label.value = "??"

def click_handler(event=None, feature=None, id=None, properties=None):
    dep = feature["properties"]["code"]
    dep_name = feature["properties"]["nom"]
    incidence = get_incidence(dep)
    positivity = get_positivity(dep)
    label.value = (f"{dep_name} ({dep}) : taux d'incidence = {incidence:.0f}, "
    f"taux de positivité = {positivity:.2f} %")
    
    
geo_json.on_click(click_handler)

m.add_layer(geo_json)

out = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out:
    display(cmap)
    
widget_control = ipyl.WidgetControl(widget=out, position='topright')
m.add_control(widget_control)

out2 = ipyw.widgets.Output(layout={'border': '1px solid black'})
with out2:
    display(label)

widget_control2 = ipyl.WidgetControl(widget=out2, position='topleft')
m.add_control(widget_control2)
m

## Taux de positivité et d'incidence pour les départements à forte dynamique épidémique

Il est intéressant d'étudier plus en détail le cas des départements pour lesquels l'épidémie est la plus forte. Ici, on considère d'abord les départements métropolitains pour lesquels le taux d'incidence est supérieur à 150. La figure à gauche représente le taux de positivité sur 7 jours pour ces départements à forte épidémie.
Les taux d'incidence sont tracés sur la figure de droite.
Le niveau d'incidence égale à 50 est considéré comme un seuil d'alerte (ligne horizontale noire sur la figure).

In [ ]:
plot_fast_dyn = PlotFasterDyn(min_incidence=200)

In [ ]:
plot_fast_dyn.layout_inputs

In [ ]:
plot_fast_dyn.plot()

Attention, le taux d'incidence peut baisser juste parce qu'on teste moins. C'est ce qui se passe début septembre pour certains départements (par exemple les Bouches-du-Rhône) avec une diminution du taux d'incidence mais une augmentation du taux de positivité!

## Dynamique sur quelques jours par départements 

Il est aussi intéressant de représenter l'incidence en fonction du nombre de tests normalisé, le ratio entre ces deux valeurs étant le taux de positivité (droites et pourcentages rouges). Un département est représenté par cinq points correspondant à 5 jours.

In [ ]:
state_incidence_vs_tests = StatePlotIncidenceVersusTests(min_incidence=200)

In [ ]:
state_incidence_vs_tests.widget_date

In [ ]:
state_incidence_vs_tests.layout_inputs

In [ ]:
state_incidence_vs_tests.plot()

## Taux de positivité pour un département décomposé en classes d'âges


In [ ]:
summary = Summary1loc(loc="38")

In [ ]:
summary.widget_dep

In [ ]:
summary.widget_yscale

In [ ]:
summary.plot()

In [ ]:
summary = Summary1loc(loc="75")

In [ ]:
summary.widget_dep

In [ ]:
summary.widget_yscale

In [ ]:
summary.plot()

Isère : on observe une accélération très rapide (super exponentielle) de l'épidémie dans les premiers jours de septembre (temps de doublement de moins de 10 jours). 